In [1]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker
faker = Faker()

# Connect to SQLite database (or create it)
conn = sqlite3.connect(r'V:\Drive D\DOWNLOAD\ML\Capstone2\user1.db')
cursor = conn.cursor()

# Create the tables
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT,
        last_name TEXT,
        email TEXT UNIQUE,
        phone_number TEXT,
        address TEXT,
        date_of_birth DATE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    CREATE TABLE IF NOT EXISTS Accounts (
        account_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        account_number TEXT UNIQUE,
        account_type TEXT,
        balance REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        account_id INTEGER,
        transaction_type TEXT,
        amount REAL,
        description TEXT,
        transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT,
        FOREIGN KEY (account_id) REFERENCES Accounts(account_id)
    );

    CREATE TABLE IF NOT EXISTS Cards (
        card_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        card_number TEXT UNIQUE,
        card_type TEXT,
        expiry_date DATE,
        status TEXT,
        pin TEXT,
        credit_limit REAL,
        current_balance REAL,
        rewards_balance REAL,
        international_usage BOOLEAN DEFAULT FALSE,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Loans (
        loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        loan_type TEXT,
        principal_amount REAL,
        interest_rate REAL,
        emi_amount REAL,
        loan_term INTEGER, -- in months
        start_date DATE,
        end_date DATE,
        status TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Statements (
        statement_id INTEGER PRIMARY KEY AUTOINCREMENT,
        account_id INTEGER,
        start_date DATE,
        end_date DATE,
        statement_type TEXT,
        statement_pdf BLOB,
        FOREIGN KEY (account_id) REFERENCES Accounts(account_id)
    );

    CREATE TABLE IF NOT EXISTS Alerts (
        alert_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        alert_type TEXT,
        threshold_amount REAL,
        alert_frequency TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Security (
        security_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        password_hash TEXT,
        two_factor_enabled BOOLEAN DEFAULT FALSE,
        last_password_change DATE,
        account_locked BOOLEAN DEFAULT FALSE,
        lock_reason TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS CustomerSupport (
        support_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        query_type TEXT,
        query_description TEXT,
        query_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Analytics (
        analytics_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        spending_analysis TEXT,
        savings_goals TEXT,
        interest_earned REAL,
        analysis_date DATE,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );
''')

# Insert one user into the Users table
user_data = (
    faker.first_name(),
    faker.last_name(),
    faker.email(),
    faker.phone_number(),
    faker.address(),
    faker.date_of_birth(tzinfo=None, minimum_age=18, maximum_age=65).strftime('%Y-%m-%d')
)
cursor.execute('''
    INSERT INTO Users (first_name, last_name, email, phone_number, address, date_of_birth)
    VALUES (?, ?, ?, ?, ?, ?)
''', user_data)
user_id = cursor.lastrowid

# Insert accounts for the user
account_types = ['Savings', 'Chequing', 'Credit', 'Loan']
account_data = []
for account_type in account_types:
    account_data.append((
        user_id,
        faker.bban(),  # Random account number
        account_type,
        round(random.uniform(1000, 100000), 2),
        'Active'
    ))

cursor.executemany('''
    INSERT INTO Accounts (user_id, account_number, account_type, balance, status)
    VALUES (?, ?, ?, ?, ?)
''', account_data)

# Insert transactions for the user
transaction_data = []
for account_id in range(1, len(account_data) + 1):
    for _ in range(random.randint(5, 15)):
        transaction_data.append((
            account_id,
            random.choice(['Deposit', 'Withdrawal', 'Transfer', 'Payment']),
            round(random.uniform(10, 5000), 2),
            faker.sentence(),
            'Completed'
        ))

cursor.executemany('''
    INSERT INTO Transactions (account_id, transaction_type, amount, description, status)
    VALUES (?, ?, ?, ?, ?)
''', transaction_data)

# Insert cards for the user
card_data = []
for _ in range(2):  # Assume 1 debit and 1 credit card
    card_data.append((
        user_id,
        faker.credit_card_number(),
        random.choice(['Credit', 'Debit']),
        faker.date_between(start_date='today', end_date='+3y').strftime('%Y-%m-%d'),
        'Active',
        faker.random_number(digits=4),
        round(random.uniform(10000, 50000), 2),  # credit limit
        round(random.uniform(0, 10000), 2),  # current balance
        round(random.uniform(0, 1000), 2),  # rewards balance
        random.choice([True, False])
    ))

cursor.executemany('''
    INSERT INTO Cards (user_id, card_number, card_type, expiry_date, status, pin, credit_limit, current_balance, rewards_balance, international_usage)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', card_data)

# Insert loans for the user
loan_data = []
for _ in range(1):  # Assume 1 loan
    principal = round(random.uniform(5000, 50000), 2)
    interest_rate = round(random.uniform(3.0, 15.0), 2)
    loan_term = random.randint(12, 60)  # 1 to 5 years
    emi = round((principal + (principal * interest_rate / 100)) / loan_term, 2)
    loan_data.append((
        user_id,
        random.choice(['Personal', 'Mortgage', 'Auto', 'Student', 'Business']),
        principal,
        interest_rate,
        emi,
        loan_term,
        faker.date_between(start_date='-2y', end_date='today').strftime('%Y-%m-%d'),
        faker.date_between(start_date='today', end_date='+5y').strftime('%Y-%m-%d'),
        'Active'
    ))

cursor.executemany('''
    INSERT INTO Loans (user_id, loan_type, principal_amount, interest_rate, emi_amount, loan_term, start_date, end_date, status)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', loan_data)

# Insert a few alerts for the user
alert_data = []
for _ in range(3):
    alert_data.append((
        user_id,
        random.choice(['Balance', 'Transaction', 'Payment Due', 'Custom']),
        round(random.uniform(100, 1000), 2),
        random.choice(['Daily', 'Weekly', 'Monthly', 'Instant'])
    ))

cursor.executemany('''
    INSERT INTO Alerts (user_id, alert_type, threshold_amount, alert_frequency)
    VALUES (?, ?, ?, ?)
''', alert_data)

# Insert security details for the user
security_data = (
    user_id,
    faker.password(),
    random.choice([True, False]),
    faker.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d'),
    random.choice([True, False]),
    faker.sentence() if random.choice([True, False]) else None
)
cursor.execute('''
    INSERT INTO Security (user_id, password_hash, two_factor_enabled, last_password_change, account_locked, lock_reason)
    VALUES (?, ?, ?, ?, ?, ?)
''', security_data)

# Insert customer support queries for the user
support_data = []
for _ in range(2):
    support_data.append((
        user_id,
        random.choice(['General', 'Technical', 'Billing', 'Card Issue', 'Loan Issue', 'Other']),
        faker.sentence(),
        'Open'
    ))

cursor.executemany('''
    INSERT INTO CustomerSupport (user_id, query_type, query_description, status)
    VALUES (?, ?, ?, ?)
''', support_data)

# Insert analytics data for the user
analytics_data = (
    user_id,
    '{}',  # Assuming JSON format for spending analysis
    '{}',  # Assuming JSON format for savings goals
    round(random.uniform(100, 10000), 2),
    faker.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d')
)
cursor.execute('''
    INSERT INTO Analytics (user_id, spending_analysis, savings_goals, interest_earned, analysis_date)
    VALUES (?, ?, ?, ?, ?)
''', analytics_data)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database created and filled with random data for one user.")


Database created and filled with random data for one user.


In [3]:
import sqlite3
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker
faker = Faker()

# Connect to SQLite database (or create it)
conn = sqlite3.connect(r'V:\Drive D\DOWNLOAD\ML\Capstone2\user2.db')
cursor = conn.cursor()

# Create the tables
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT,
        last_name TEXT,
        email TEXT UNIQUE,
        phone_number TEXT,
        address TEXT,
        date_of_birth DATE,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    CREATE TABLE IF NOT EXISTS Accounts (
        account_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        account_number TEXT UNIQUE,
        account_type TEXT,
        balance REAL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        account_id INTEGER,
        transaction_type TEXT,
        amount REAL,
        description TEXT,
        transaction_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        status TEXT,
        FOREIGN KEY (account_id) REFERENCES Accounts(account_id)
    );

    CREATE TABLE IF NOT EXISTS Cards (
        card_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        card_number TEXT UNIQUE,
        card_type TEXT,
        expiry_date DATE,
        status TEXT,
        pin TEXT,
        credit_limit REAL,
        current_balance REAL,
        rewards_balance REAL,
        international_usage BOOLEAN DEFAULT FALSE,
        total_paid REAL DEFAULT 0,
        remaining_balance REAL,
        last_payment_date DATE,
        last_payment_amount REAL,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Loans (
        loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        loan_type TEXT,
        principal_amount REAL,
        interest_rate REAL,
        emi_amount REAL,
        loan_term INTEGER, -- in months
        start_date DATE,
        end_date DATE,
        status TEXT,
        total_paid REAL DEFAULT 0,
        remaining_balance REAL,
        last_payment_date DATE,
        last_payment_amount REAL,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Alerts (
        alert_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        alert_type TEXT,
        threshold_amount REAL,
        alert_frequency TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Security (
        security_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        password_hash TEXT,
        two_factor_enabled BOOLEAN DEFAULT FALSE,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );
''')

# Insert random data for one user
user_data = (
    faker.first_name(),
    faker.last_name(),
    faker.email(),
    faker.phone_number(),
    faker.address(),
    faker.date_of_birth(minimum_age=18, maximum_age=90)
)
cursor.execute('''
    INSERT INTO Users (first_name, last_name, email, phone_number, address, date_of_birth)
    VALUES (?, ?, ?, ?, ?, ?)
''', user_data)
user_id = cursor.lastrowid

# Insert random account data
account_data = (
    user_id,
    faker.bban(),
    random.choice(['Savings', 'Checking']),
    round(random.uniform(1000, 50000), 2),
    random.choice(['Active', 'Inactive'])
)
cursor.execute('''
    INSERT INTO Accounts (user_id, account_number, account_type, balance, status)
    VALUES (?, ?, ?, ?, ?)
''', account_data)
account_id = cursor.lastrowid

# Insert random transactions
for _ in range(10):
    transaction_data = (
        account_id,
        random.choice(['Deposit', 'Withdrawal', 'Transfer', 'Payment']),
        round(random.uniform(10, 5000), 2),
        faker.sentence(),
        random.choice(['Completed', 'Pending', 'Failed'])
    )
    cursor.execute('''
        INSERT INTO Transactions (account_id, transaction_type, amount, description, status)
        VALUES (?, ?, ?, ?, ?)
    ''', transaction_data)

# Insert random card data
card_data = (
    user_id,
    faker.credit_card_number(),
    random.choice(['Credit', 'Debit']),
    faker.date_between(start_date='-5y', end_date='+5y'),
    random.choice(['Active', 'Blocked']),
    faker.password(length=4, special_chars=False, digits=True, upper_case=False, lower_case=False),
    round(random.uniform(1000, 20000), 2),
    round(random.uniform(0, 10000), 2),
    round(random.uniform(0, 1000), 2),
    round(random.uniform(0, 5000), 2),
    round(random.uniform(0, 5000), 2),
    faker.date_between(start_date='-1y', end_date='today')
)
cursor.execute('''
    INSERT INTO Cards (user_id, card_number, card_type, expiry_date, status, pin, credit_limit, 
                       current_balance, rewards_balance, total_paid, last_payment_amount, last_payment_date)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', card_data)

# Insert random loan data
loan_data = (
    user_id,
    random.choice(['Home Loan', 'Car Loan', 'Personal Loan']),
    round(random.uniform(10000, 500000), 2),
    round(random.uniform(3.0, 10.0), 2),
    round(random.uniform(100, 5000), 2),
    random.randint(12, 360),  # Loan term in months
    faker.date_between(start_date='-5y', end_date='today'),
    faker.date_between(start_date='today', end_date='+30y'),
    random.choice(['Active', 'Closed', 'Defaulted']),
    round(random.uniform(1000, 20000), 2),
    round(random.uniform(1000, 50000), 2),
    faker.date_between(start_date='-1y', end_date='today'),
    round(random.uniform(100, 5000), 2)
)
cursor.execute('''
    INSERT INTO Loans (user_id, loan_type, principal_amount, interest_rate, emi_amount, loan_term, start_date, 
                       end_date, status, total_paid, remaining_balance, last_payment_date, last_payment_amount)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', loan_data)

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Database created and filled with random data for one user.")


Database created and filled with random data for one user.


In [4]:
import sqlite3
from faker import Faker
import random

# Initialize Faker
faker = Faker()

# Connect to SQLite database (or create it)
conn = sqlite3.connect('banking_backend.db')
cursor = conn.cursor()

# Drop the existing tables if they exist
cursor.executescript('''
    DROP TABLE IF EXISTS Users;
    DROP TABLE IF EXISTS Accounts;
    DROP TABLE IF EXISTS DebitCards;
    DROP TABLE IF EXISTS CreditCards;
    DROP TABLE IF EXISTS Loans;
    DROP TABLE IF EXISTS Transactions;
    DROP TABLE IF EXISTS CardPayments;
    DROP TABLE IF EXISTS CustomerServiceActions;
''')

# Create the updated tables
cursor.executescript('''
    CREATE TABLE IF NOT EXISTS Users (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT,
        last_name TEXT,
        email TEXT,
        phone_number TEXT,
        address TEXT,
        created_at DATE
    );

    CREATE TABLE IF NOT EXISTS Accounts (
        account_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        account_number TEXT UNIQUE,
        account_type TEXT,
        balance REAL,
        created_at DATE,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS DebitCards (
        debit_card_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        card_number TEXT UNIQUE,
        expiry_date DATE,
        status TEXT,
        pin TEXT,
        balance REAL,
        last_transaction_date DATE,
        last_payment_amount REAL,
        international_usage BOOLEAN DEFAULT FALSE,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS CreditCards (
        credit_card_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        card_number TEXT UNIQUE,
        expiry_date DATE,
        status TEXT,
        pin TEXT,
        credit_limit REAL,
        current_balance REAL,
        rewards_balance REAL,
        international_usage BOOLEAN DEFAULT FALSE,
        total_paid REAL DEFAULT 0,
        remaining_balance REAL,
        last_payment_date DATE,
        last_payment_amount REAL,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Loans (
        loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        loan_type TEXT,
        principal_amount REAL,
        interest_rate REAL,
        term_months INTEGER,
        start_date DATE,
        end_date DATE,
        last_payment_date DATE,
        last_payment_amount REAL,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );

    CREATE TABLE IF NOT EXISTS Transactions (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        account_id INTEGER,
        card_id INTEGER,
        card_type TEXT,
        amount REAL,
        transaction_type TEXT,
        transaction_date DATE,
        description TEXT,
        FOREIGN KEY (account_id) REFERENCES Accounts(account_id)
            ON DELETE CASCADE,
        FOREIGN KEY (card_id) REFERENCES DebitCards(debit_card_id)
            ON DELETE CASCADE,
        FOREIGN KEY (card_id) REFERENCES CreditCards(credit_card_id)
            ON DELETE CASCADE
    );

    CREATE TABLE IF NOT EXISTS CardPayments (
        payment_id INTEGER PRIMARY KEY AUTOINCREMENT,
        card_id INTEGER,
        card_type TEXT,
        payment_date DATE,
        payment_amount REAL,
        FOREIGN KEY (card_id) REFERENCES DebitCards(debit_card_id)
            ON DELETE CASCADE,
        FOREIGN KEY (card_id) REFERENCES CreditCards(credit_card_id)
            ON DELETE CASCADE
    );

    CREATE TABLE IF NOT EXISTS CustomerServiceActions (
        action_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        action_type TEXT,
        action_date DATE,
        notes TEXT,
        FOREIGN KEY (user_id) REFERENCES Users(user_id)
    );
''')

# Function to populate tables with random data for one user
def populate_data():
    # Insert one user
    cursor.execute('''
        INSERT INTO Users (first_name, last_name, email, phone_number, address, created_at)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (faker.first_name(), faker.last_name(), faker.email(), faker.phone_number(), faker.address(), faker.date_this_decade()))
    
    user_id = cursor.lastrowid  # Get the ID of the inserted user

    # Insert one account for the user
    cursor.execute('''
        INSERT INTO Accounts (user_id, account_number, account_type, balance, created_at)
        VALUES (?, ?, ?, ?, ?)
    ''', (user_id, faker.uuid4(), 'savings', random.uniform(1000.00, 5000.00), faker.date_this_decade()))
    
    account_id = cursor.lastrowid  # Get the ID of the inserted account

    # Insert one debit card for the user
    cursor.execute('''
        INSERT INTO DebitCards (user_id, card_number, expiry_date, status, pin, balance, last_transaction_date, last_payment_amount, international_usage)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (user_id, faker.credit_card_number(), faker.future_date(), 'active', faker.credit_card_security_code(), random.uniform(100.00, 500.00), faker.date_this_decade(), random.uniform(10.00, 50.00), faker.boolean()))
    
    debit_card_id = cursor.lastrowid  # Get the ID of the inserted debit card

    # Insert one credit card for the user
    cursor.execute('''
        INSERT INTO CreditCards (user_id, card_number, expiry_date, status, pin, credit_limit, current_balance, rewards_balance, international_usage, total_paid, remaining_balance, last_payment_date, last_payment_amount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (user_id, faker.credit_card_number(), faker.future_date(), 'active', faker.credit_card_security_code(), random.uniform(1000.00, 5000.00), random.uniform(100.00, 500.00), random.uniform(0.00, 100.00), faker.boolean(), random.uniform(0.00, 500.00), random.uniform(100.00, 500.00), faker.date_this_decade(), random.uniform(10.00, 50.00)))
    
    credit_card_id = cursor.lastrowid  # Get the ID of the inserted credit card

    # Insert one loan for the user
    cursor.execute('''
        INSERT INTO Loans (user_id, loan_type, principal_amount, interest_rate, term_months, start_date, end_date, last_payment_date, last_payment_amount)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (user_id, 'personal', random.uniform(5000.00, 20000.00), random.uniform(2.00, 10.00), random.randint(12, 60), faker.past_date(), faker.future_date(), faker.date_this_decade(), random.uniform(100.00, 500.00)))
    
    loan_id = cursor.lastrowid  # Get the ID of the inserted loan

    # Insert transactions for the user
    for _ in range(5):  # Fewer transactions for simplicity
        cursor.execute('''
            INSERT INTO Transactions (account_id, card_id, card_type, amount, transaction_type, transaction_date, description)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (account_id, debit_card_id, 'debit', random.uniform(10.00, 200.00), random.choice(['withdrawal', 'deposit', 'payment']), faker.date_this_decade(), faker.text()))

    for _ in range(5):  # Fewer transactions for simplicity
        cursor.execute('''
            INSERT INTO Transactions (account_id, card_id, card_type, amount, transaction_type, transaction_date, description)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (account_id, credit_card_id, 'credit', random.uniform(10.00, 200.00), random.choice(['withdrawal', 'deposit', 'payment']), faker.date_this_decade(), faker.text()))

    # Insert card payments
    for _ in range(5):
        cursor.execute('''
            INSERT INTO CardPayments (card_id, card_type, payment_date, payment_amount)
            VALUES (?, ?, ?, ?)
        ''', (debit_card_id, 'debit', faker.date_this_decade(), random.uniform(10.00, 200.00)))
    
    for _ in range(5):
        cursor.execute('''
            INSERT INTO CardPayments (card_id, card_type, payment_date, payment_amount)
            VALUES (?, ?, ?, ?)
        ''', (credit_card_id, 'credit', faker.date_this_decade(), random.uniform(10.00, 200.00)))

    # Insert customer service actions
    for _ in range(5):
        cursor.execute('''
            INSERT INTO CustomerServiceActions (user_id, action_type, action_date, notes)
            VALUES (?, ?, ?, ?)
        ''', (user_id, random.choice(['block_card', 'change_pin', 'reset_password']), faker.date_this_decade(), faker.text()))

    # Commit changes and close the connection
    conn.commit()
    conn.close()

# Run the function to populate data
populate_data()


In [7]:
import sqlite3
from datetime import datetime

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('banking_backend.db')
cursor = conn.cursor()

# Insert sample data into Accounts table
cursor.execute('''
    INSERT INTO Accounts (user_id, account_number, account_type, balance, created_at)
    VALUES (1, 'SAV123456789', 'savings', 3500.75, ?)
''', (datetime.now().strftime('%Y-%m-%d'),))

cursor.execute('''
    INSERT INTO Accounts (user_id, account_number, account_type, balance, created_at)
    VALUES (1, 'CHK987654321', 'chequing', 1500.50, ?)
''', (datetime.now().strftime('%Y-%m-%d'),))



# Commit the changes and close the connection
conn.commit()
conn.close()


In [11]:
import re
from sqlalchemy import create_engine, Table, MetaData, Column, Integer, Text, Float, Date, update, select
from sqlalchemy.orm import sessionmaker

# Initialize the database connection
DATABASE_URL = "sqlite:///user1.db"  # Replace with your database URL
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

# Initialize MetaData and Table
metadata = MetaData()
accounts = Table('Accounts', metadata,
    Column('account_id', Integer, primary_key=True),
    Column('user_id', Integer),
    Column('account_number', Text, unique=True),
    Column('account_type', Text),
    Column('balance', Float),
    Column('created_at', Date),
    autoload_with=engine
)

# Map account types to account numbers
account_type_to_number = {
    'chequing': 'chequing_account_number',
    'savings': 'savings_account_number'
}

def extract_transfer_details(user_input):
    # Regex pattern to extract amount and account types
    pattern = r"transfer (\d+\.?\d*) from (\w+) to (\w+)"
    match = re.search(pattern, user_input, re.IGNORECASE)
    
    if match:
        amount = float(match.group(1))
        source_type = match.group(2).lower()
        destination_type = match.group(3).lower()
        
        # Map account types to actual account numbers
        source_account_number = account_type_to_number.get(source_type)
        destination_account_number = account_type_to_number.get(destination_type)
        
        return source_account_number, destination_account_number, amount
    else:
        return None, None, None

def transfer_funds(user_input):
    source_account_number, destination_account_number, amount = extract_transfer_details(user_input)
    
    if not source_account_number or not destination_account_number or amount is None:
        return "Invalid input. Please provide the transfer details in the correct format."

    try:
        with engine.connect() as conn:
            # Check if both accounts exist
            source_query = select([accounts]).where(accounts.c.account_number == source_account_number)
            destination_query = select([accounts]).where(accounts.c.account_number == destination_account_number)
            
            source_account = conn.execute(source_query).fetchone()
            destination_account = conn.execute(destination_query).fetchone()

            if not source_account or not destination_account:
                return "One or both account types are invalid."

            # Check if source account has sufficient balance
            if source_account['balance'] < amount:
                return "Insufficient funds in the source account."

            # Perform the transfer
            with conn.begin() as transaction:
                # Deduct amount from source account
                update_source = update(accounts).where(accounts.c.account_number == source_account_number).values(balance=accounts.c.balance - amount)
                conn.execute(update_source)

                # Add amount to destination account
                update_destination = update(accounts).where(accounts.c.account_number == destination_account_number).values(balance=accounts.c.balance + amount)
                conn.execute(update_destination)

                return "Transfer successful."

    except Exception as e:
        return f"An error occurred: {e}"

# Example usage:
user_input = "Transfer 100.00 from chequing to savings"
print(transfer_funds(user_input))


An error occurred: Column expression, FROM clause, or other columns clause element expected, got [Table('Accounts', MetaData(), Column('account_id', Integer(), table=<Accounts>, primary_key=True, nullable=False), Column('user_id', Integer(), table=<Accounts>), Column('account_number', Text(), table=<Accounts>), Column('account_type', Text(), table=<Accounts>), Column('balance', Float(), table=<Accounts>), Column('created_at', Date(), table=<Accounts>), schema=None)]. Did you mean to say select(Table('Accounts', MetaData(), Column('account_id', Integer(), table=<Accounts>, primary_key=True, nullable=False), Column('user_id', Integer(), table=<Accounts>), Column('account_number', Text(), table=<Accounts>), Column('account_type', Text(), table=<Accounts>), Column('balance', Float(), table=<Accounts>), Column('created_at', Date(), table=<Accounts>), schema=None))?


In [17]:
import sqlite3

def transfer_money(command):
    # Connect to the SQLite database
    conn = sqlite3.connect('user1.db')
    cursor = conn.cursor()
    
    # Extract details from the command
    try:
        # Example command: "Transfer 100 from chequing to savings"
        parts = command.lower().split()
        amount = float(parts[1])  # Amount to transfer
        from_account = parts[3]  # Source account type (chequing or savings)
        to_account = parts[5]  # Destination account type (chequing or savings)
        
        # Check account types
        if from_account not in ['chequing', 'savings'] or to_account not in ['chequing', 'savings']:
            print("Invalid account type. Please specify 'chequing' or 'savings'.")
            return
        
        # Get account details from the database
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (from_account,))
        from_account_details = cursor.fetchone()
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (to_account,))
        to_account_details = cursor.fetchone()
        
        if from_account_details is None or to_account_details is None:
            print("One or both account types do not exist.")
            return
        
        from_account_id, from_balance = from_account_details
        to_account_id, to_balance = to_account_details
        
        # Check if sufficient balance is available
        if from_balance < amount:
            print("Insufficient balance in the source account.")
            return
        
        # Perform the transfer
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, from_account_id))
        cursor.execute("UPDATE Accounts SET balance = balance + ? WHERE account_id = ?", (amount, to_account_id))
        
        # Commit the transaction
        conn.commit()
        print(f"Successfully transferred {amount} from {from_account} to {to_account}.")
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        # Close the connection
        conn.close()

# Example usage
transfer_money("Transfer 100 from savings to chequing")


Successfully transferred 100.0 from savings to chequing.


In [16]:
import sqlite3

def transfer_money(command):
    # Connect to the SQLite database
    conn = sqlite3.connect('user1.db')
    cursor = conn.cursor()
    
    # Extract details from the command
    try:
        # Example command: "Transfer 100 from 123456 to 654321" or "Transfer 100 from chequing to savings"
        parts = command.lower().split()
        amount = float(parts[1])  # Amount to transfer
        
        if parts[2].isdigit():  # Check if the 'from' part is an account number
            from_account_number = parts[2]
            to_account_number = parts[4]
            
            # Get account details by account number
            cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_number = ?", (from_account_number,))
            from_account_details = cursor.fetchone()
            cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_number = ?", (to_account_number,))
            to_account_details = cursor.fetchone()
            
            if from_account_details is None or to_account_details is None:
                print("One or both account numbers do not exist.")
                return
            
            from_account_id, from_balance = from_account_details
            to_account_id, to_balance = to_account_details

        else:  # Handle cases where account types are provided
            from_account_type = parts[2]
            to_account_type = parts[4]
            
            # Get account details by type
            cursor.execute("SELECT account_id, account_number, balance FROM Accounts WHERE account_type = ?", (from_account_type,))
            from_account_details = cursor.fetchone()
            cursor.execute("SELECT account_id, account_number, balance FROM Accounts WHERE account_type = ?", (to_account_type,))
            to_account_details = cursor.fetchone()
            
            if from_account_details is None or to_account_details is None:
                print("One or both account types do not exist.")
                return
            
            from_account_id, from_account_number, from_balance = from_account_details
            to_account_id, to_account_number, to_balance = to_account_details

        # Check if sufficient balance is available
        if from_balance < amount:
            print("Insufficient balance in the source account.")
            return
        
        # Perform the transfer
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, from_account_id))
        cursor.execute("UPDATE Accounts SET balance = balance + ? WHERE account_id = ?", (amount, to_account_id))
        
        # Commit the transaction
        conn.commit()
        print(f"Successfully transferred {amount} from account {from_account_number} to account {to_account_number}.")
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        # Close the connection
        conn.close()

# Example usage
transfer_money("Transfer 100 from CHK987654321 to SAV123456789")



One or both account types do not exist.


In [18]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('user1.db')
cursor = conn.cursor()

# Step 1: Add the 'autodebit' column to the CreditCards table
cursor.execute("""
    ALTER TABLE CreditCards
    ADD COLUMN autodebit INTEGER DEFAULT 0;
""")

# Step 2: Update the first row to have 'autodebit' set to 1
cursor.execute("""
    UPDATE CreditCards
    SET autodebit = 1
    WHERE credit_card_id = (SELECT MIN(credit_card_id) FROM CreditCards);
""")

# Commit the changes and close the connection
conn.commit()
conn.close()


In [20]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('user1.db')

# Create a cursor object
cur = conn.cursor()

# Add a new column 'remaining_balance' to the 'Loans' table
cur.execute("ALTER TABLE Loans ADD COLUMN remaining_balance REAL")

# Update the 'remaining_balance' column with some amount, let's say 1000.00
cur.execute("UPDATE Loans SET remaining_balance = 1000.00")

# Commit the changes and close the connection
conn.commit()
conn.close()


In [37]:
import sqlite3

def pay_loan_emi(command, db_file):
    conn = sqlite3.connect('user1.db')
    cursor = conn.cursor()

    try:
        # Example command: "Pay 500.00 for home loan EMI from savings"
        parts = command.lower().split()
        amount = float(parts[1])  # Amount to pay
        loan_type = parts[3]  # Type of loan (home, personal, etc.)
        from_account = parts[-1]  # Source account type (chequing or savings)
        
        # Get account and loan details from the database
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (from_account,))
        account_details = cursor.fetchone()
        cursor.execute("SELECT loan_id, remaining_balance FROM Loans WHERE loan_type = ?", (loan_type,))
        loan_details = cursor.fetchone()
        
        if account_details is None or loan_details is None:
            return "Invalid account or loan details."
        
        account_id, account_balance = account_details
        loan_id, loan_balance = loan_details
        
        # Check if sufficient balance is available
        if account_balance < amount:
            return "Insufficient balance in the source account."
        
        # Perform the payment
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, account_id))
        cursor.execute("UPDATE Loans SET remaining_balance = remaining_balance - ? WHERE loan_id = ?", (amount, loan_id))
        
        conn.commit()
        return f"Successfully paid {amount} towards the {loan_type} loan EMI from your {from_account} account."
    
    except Exception as e:
        return f"An error occurred: {e}"
    
    finally:
        conn.close()

# Test the function
print(pay_loan_emi("Pay 00.00 for personal loan EMI from savings", 'customer.db'))


Successfully paid 0.0 towards the personal loan EMI from your savings account.


In [42]:

def pay_credit_card_bill(command):
    conn = sqlite3.connect('user1.db')
    cursor = conn.cursor()
    try:
        # Example command: "Pay 150.00 for credit card ending in 1234 from chequing"
        parts = command.lower().split()
        amount = float(parts[1])  # Amount to pay
        card_number = parts[3]  # Last 4 digits of the card
        from_account = parts[-1]  # Source account type (chequing or savings)
        
        # Get account and credit card details from the database
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (from_account,))
        account_details = cursor.fetchone()
        cursor.execute("SELECT credit_card_id, current_balance FROM CreditCards WHERE card_number= ?", (card_number,))
        card_details = cursor.fetchone()
        
        if account_details is None or card_details is None:
            return "Invalid account or credit card details."
        
        account_id, account_balance = account_details
        credit_card_id, card_balance = card_details
        
        # Check if sufficient balance is available
        if account_balance < amount:
            return "Insufficient balance in the source account."
        
        # Perform the payment
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, account_id))
        cursor.execute("UPDATE CreditCards SET current_balance = current_balance - ? WHERE credit_card_id = ?", (amount, credit_card_id))
        
        conn.commit()
        return f"Successfully paid {amount} towards the credit card ending in {card_number} from your {from_account} account."
    
    except Exception as e:
        return f"An error occurred: {e}"
    
    finally:
        conn.close()
        
print(pay_credit_card_bill("Pay 75.00 for credit card ending in 4117959262411590115 from chequing"))


Invalid account or credit card details.


In [47]:
import sqlite3
import re

def pay_credit_card_bill(command):
    conn = sqlite3.connect('user1.db')
    cursor = conn.cursor()
    try:
        # Example command: "Pay 150.00 for credit card ending in 4117959262411590115 from chequing"
        # Improved parsing with regex
        pattern = r"Pay (\d+\.?\d*) for credit card number (\d+) from (\w+)"
        match = re.search(pattern, command, re.IGNORECASE)
        
        if not match:
            return "Invalid command format."

        amount = float(match.group(1))
        card_number = match.group(2)
        from_account = match.group(3)

        # Debug prints
        print(f"Amount: {amount}")
        print(f"Card number: {card_number}")
        print(f"From account: {from_account}")

        # Get account and credit card details from the database
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (from_account,))
        account_details = cursor.fetchone()
        cursor.execute("SELECT credit_card_id, current_balance FROM CreditCards WHERE card_number = ?", (card_number,))
        card_details = cursor.fetchone()

        # Debug prints
        print(f"Account details: {account_details}")
        print(f"Card details: {card_details}")

        if account_details is None or card_details is None:
            return "Invalid account or credit card details."

        account_id, account_balance = account_details
        credit_card_id, card_balance = card_details

        # Check if sufficient balance is available
        if account_balance < amount:
            return "Insufficient balance in the source account."

        # Perform the payment
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, account_id))
        cursor.execute("UPDATE CreditCards SET current_balance = current_balance - ? WHERE credit_card_id = ?", (amount, credit_card_id))

        conn.commit()
        return f"Successfully paid {amount} towards the credit card ending in {card_number} from your {from_account} account."

    except Exception as e:
        return f"An error occurred: {e}"

    finally:
        conn.close()

# Example call to the function
print(pay_credit_card_bill("Pay 75.00 for credit card number 4117959262411590115 from chequing"))


Amount: 75.0
Card number: 4117959262411590115
From account: chequing
Account details: (3, 1705.5)
Card details: (1, 333.27275706280983)
Successfully paid 75.0 towards the credit card ending in 4117959262411590115 from your chequing account.


In [48]:
import sqlite3
import re

def pay_credit_card_bill(command):
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()
    try:
        # Regex pattern to match the command format with "number" instead of "ending in"
        pattern = r'^Pay\s+(\d+\.?\d*)\s+for\s+credit\s+card\s+number\s+(\d+)\s+from\s+(\w+)$'
        match = re.search(pattern, command, re.IGNORECASE)
        
        if not match:
            return "Invalid command format."

        amount = float(match.group(1))
        card_number = match.group(2)
        from_account = match.group(3)

        # Debug prints
        print(f"Amount: {amount}")
        print(f"Card number: {card_number}")
        print(f"From account: {from_account}")

        # Get account and credit card details from the database
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (from_account,))
        account_details = cursor.fetchone()
        cursor.execute("SELECT credit_card_id, current_balance FROM CreditCards WHERE card_number = ?", (card_number,))
        card_details = cursor.fetchone()

        # Debug prints
        print(f"Account details: {account_details}")
        print(f"Card details: {card_details}")

        if account_details is None or card_details is None:
            return "Invalid account or credit card details."

        account_id, account_balance = account_details
        credit_card_id, card_balance = card_details

        # Check if sufficient balance is available
        if account_balance < amount:
            return "Insufficient balance in the source account."

        # Perform the payment
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, account_id))
        cursor.execute("UPDATE CreditCards SET current_balance = current_balance - ? WHERE credit_card_id = ?", (amount, credit_card_id))

        conn.commit()
        return f"Successfully paid {amount} towards the credit card number {card_number} from your {from_account} account."

    except Exception as e:
        return f"An error occurred: {e}"

    finally:
        conn.close()

# Example call to the function
print(pay_credit_card_bill("Pay 75.00 for credit card number 4117959262411590115 from chequing"))


Amount: 75.0
Card number: 4117959262411590115
From account: chequing
Account details: (3, 1630.5)
Card details: (1, 258.27275706280983)
Successfully paid 75.0 towards the credit card number 4117959262411590115 from your chequing account.


In [49]:
import sqlite3
import re

def pay_credit_card_bill(command):
    # Connect to the database
    conn = sqlite3.connect('your_database_filename.db')
    cursor = conn.cursor()
    
    try:
        # Extract information from the command using regular expression
        pattern = r"Pay (\d+\.\d+) for credit card number (\d+) from (\w+)"
        match = re.search(pattern, command)
        
        if not match:
            return "Invalid command format."
        
        amount = float(match.group(1))
        card_number = match.group(2)
        from_account = match.group(3)

        # Debug prints
        print(f"Amount: {amount}")
        print(f"Card number: {card_number}")
        print(f"From account: {from_account}")

        # Get account and credit card details from the database
        cursor.execute("SELECT account_id, balance FROM Accounts WHERE account_type = ?", (from_account,))
        account_details = cursor.fetchone()
        cursor.execute("SELECT credit_card_id, current_balance FROM CreditCards WHERE card_number = ?", (card_number,))
        card_details = cursor.fetchone()

        # Debug prints
        print(f"Account details: {account_details}")
        print(f"Card details: {card_details}")

        if account_details is None or card_details is None:
            return "Invalid account or credit card details."

        account_id, account_balance = account_details
        credit_card_id, card_balance = card_details

        # Check if sufficient balance is available
        if account_balance < amount:
            return "Insufficient balance in the source account."

        # Perform the payment
        cursor.execute("UPDATE Accounts SET balance = balance - ? WHERE account_id = ?", (amount, account_id))
        cursor.execute("UPDATE CreditCards SET current_balance = current_balance - ? WHERE credit_card_id = ?", (amount, credit_card_id))

        conn.commit()
        return f"Successfully paid {amount} towards the credit card number {card_number} from your {from_account} account."

    except Exception as e:
        return f"An error occurred: {e}"

    finally:
        conn.close()

# Example call to the function
print(pay_credit_card_bill("Pay 75.00 for credit card number 4117959262411590115 from chequing"))


Amount: 75.0
Card number: 4117959262411590115
From account: chequing
An error occurred: no such table: Accounts


In [2]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('user1.db')  # Replace with your database file
cursor = conn.cursor()

# SQL command to add a new column to the CreditCards table
add_column_sql = '''
ALTER TABLE CreditCards
ADD COLUMN is_blocked INTEGER DEFAULT 0;
'''

# Execute the SQL command
try:
    cursor.execute(add_column_sql)
    print("Column 'is_blocked' added successfully.")
except sqlite3.OperationalError as e:
    print(f"Error: {e}")

# Commit changes and close the connection
conn.commit()
conn.close()


Column 'is_blocked' added successfully.


In [4]:
pip install openpyxl

  Using cached et_xmlfile-1.1.0-py3-none-any.whl.metadata (1.8 kB)
Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.
